## Instalando e importando as bibliotecas

```shell
pip3 install bs4 --user
```

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import warnings
warnings.filterwarnings("ignore")

Agora, vamos tentar baixar a lista de vôos entre São Paulo e Rio do site Kayak. Olhando para a estrutura HTML, vemos que cada palestra, ou a linha do dataset que quero montar, é um elemento "tr" com a classe "data"
<img src='img/kayak-inspected.png'/>

### Vamos tentar capturar esta informação por meio do Beautiful Soup.

In [ ]:
# Definimos a URL
# url = 'https://www.festivalpath.com.br/programacao/palestra?data=sabado--1-de-junho'
url = 'https://www.kayak.com/flights/SAO-RIO/2020-12-07/2020-12-12?sort=bestflight_a'

userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
]

In [ ]:
#capturamos a resposta que a URL nos trás
response = get(url, headers={"User-agent": userAgents[1]})

In [ ]:
response

O código "200" na resposta do servidor, se refere a uma resposta "OK" (ou sem erros) <br>
Para saber mais sobre os códigos de resposta, basta acessar este link:<br>
https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Status

junto com o código de resposta, a minha variável também armazenou o conteúdo da resposta do servidor, que podemos acessar por meio de: "variável.content"

In [ ]:
olhadinha=700
display(str(response.content[:olhadinha]) + '…')

O próximo passo é passar este conteúdo para o beautiful soup organizá-lo para nós.

In [ ]:
parsed_content = BeautifulSoup(response.content, 'html.parser')

In [ ]:
display(str(parsed_content)[:olhadinha] + '…')

Além de mais ingeligível, o conteúdo armazenado nesta variável "parsed_content" se torna navegável e agora podemos pedir para ele retornar o elemento "tr", de classe "data". Correspondente às linhas do data set que quero construir

In [ ]:
parsed_content.findAll('div', {'class':'Base-Results-HorizonResult'})

A resposta que obtive foi 200, então a minha consulta não apresentou um erro.<br>
Mas o resultado voltou vazio, algo está errado.<br>
Posso ter errado a URL ou a referência que tentei buscar.<br>

Se eu procurar no meu notebook (utilizando CTRL+F) pela string ' tr class="data" ', não vou encontrar nada, então de fato parte do conteúdo acabou não retornando. Vamos voltar ao site.

Se repararmos, as informações contidas na tabela demoram um pouco mais para carregar. <br>
O que está acontecendo, é que o meu código está pegando o HTML carregado no primeiro momento apenas. No caso deste site, um javascript modifica o HTML e adiciona informações ali depois que o HTML inicial já foi carregado.

Então preciso de algo que carregue a página, espere alguns segundos e apenas depois deste tempo capture as informações e as guarde em uma variável.

### Para isto, vamos utilizar o Selenium

instalando o selenium:

```shell
pip3 install selenium --user
```

No caso do selenium, não basta instalar a biblioteca, precisamos baixar um webdriver para ser utilizado pela biblioteca. Este webdriver vai abrir um browser e vai permitir que o código navegue na página conforme as nossas instruções.

### Baixando o webdriver do Google Chrome

primero passo: descubra qual é a versão do seu Chrome:

Opções ➔ Ajuda ➔ Sobre o Chrome

<img src='img/about-chrome.png'>

Vá em http://chromedriver.chromium.org/downloads e baixe o webdriver correspondente a sua versão do Chrome.

# Usando o Selenium

In [ ]:
#importamos a função webdriver da biblioteca selenium
from selenium import webdriver

Como o webdriver vai abrir um navegador, temos que tomar alguns cuidados. <br>
Por exemplo: dependendo da responsividade do site, o html pode ser diferente dependendo do tamanho da janela. <br>
Então só para garantir, vou falar para ele abrir a tela maximizada.

In [ ]:
options = webdriver.FirefoxOptions()
# options = webdriver.ChromeOptions()
# options = webdriver.SafariOptions()
options.add_argument("--start-maximized")
# options.add_argument("--display=localhost:11.0")
# options.headless = True

In [ ]:
options

Definimos a variável driver, passando os parâmetros caminho (onde o webdriver está no meu pc) e opções (definidas no passo anterior)

## Em Linux
```shell
wget "https://github.com/mozilla/geckodriver/releases/download/v0.26.0/geckodriver-v0.26.0-linux64.tar.gz"; tar -zxvf "geckodriver-v0.26.0-linux64.tar.gz"
```

## Num Mac
```shell
curl "https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-macos.tar.gz" > geckodriver-v0.28.0-macos.tar.gz; tar -zxvf geckodriver-v0.28.0-macos.tar.gz
```

In [ ]:
# path = './chromedriver.exe'
path = './geckodriver.macos'
# path = 'safaridriver'

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)
# driver = webdriver.Firefox(options=options)
# driver = webdriver.Safari()

In [ ]:
import time

In [ ]:
#isto vai abrir o navegador
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)
#isto vai fazer o navegador abrir no endereço
driver.get(url)
#o camando abaixo vai fazer o navegador esperar um pouco antes de fazer qualquer coisa
time.sleep(5)

Para navegar pelas urls em um navegador, basta seguir o exemplo abaixo:

In [ ]:
url1 = 'https://www.wikipedia.org/'
url2 = 'https://vejasp.abril.com.br'
url3 = 'https://br.digitalhouse.com/'

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)

driver.get(url1)
time.sleep(4)
driver.get(url2)
time.sleep(4)
driver.get(url3)
time.sleep(4)

A forma de capturar o HTML é através da função driver.page_source <br>
O código abaixo vai abrir o site, esperar as informações carregarem e capturar o HTML.

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(8)
#na linha abaixo guardamos o HTML final em uma variável
page_source = driver.page_source

Agora com o conteúdo em mãos, podemos passar para o Beautiful Soup

In [ ]:
page_source

In [ ]:
parsed_content = BeautifulSoup(page_source, 'html.parser')

In [ ]:
parsed_content

Agora quando pedirmos para retornar o elemento "tr" de classe "data". Ele não virá vazio.


In [ ]:
parsed_content.findAll('div', {'class':'Base-Results-HorizonResult'})

In [ ]:
voos = parsed_content.findAll('div', {'class':'Base-Results-HorizonResult'})

Repetimos o mesmo processo da aula anterior e primeiro aprendemos em como extrair as informações de uma única palestra:

In [ ]:
voos[0]

In [ ]:
primeiro_voo = voos[0]

In [ ]:
primeiro_voo.findAll('div', class_="times")

Primeira saída e chegada:

In [ ]:
primeiro_voo.findAll('div', class_="times")[0].findAll('span', class_="depart-time")[0].text, primeiro_voo.findAll('div', class_="times")[0].findAll('span', class_="arrival-time")[0].text

Loop para capturar tudo:

In [ ]:
horario_voos=[]

for v in voos:
    saida = v.findAll('div', class_="times")[0].findAll('span', class_="depart-time")[0].text
    chegada = v.findAll('div', class_="times")[0].findAll('span', class_="arrival-time")[0].text
    
    horario_voos.append({'saida': saida, 'chegada': chegada})
    
horario_voos

construindo o dataframe:

In [ ]:
import pandas as pd

In [ ]:
voos_df = pd.DataFrame(horario_voos)
voos_df

## Navegando com o Selenium

In [ ]:
url = 'https://www.zapimoveis.com.br/venda/apartamentos/sp+sao-paulo+zona-oeste+pinheiros'

Trabalhando com listas dropdown:

Algumas vezes é mais fácil navegar pelo elementos html do que pela URL, por exemplo a forma com que a URL deste site muda não se encaixa diretamente com a opção disponível na lista dropdown:

<img src='img/img_9.png'>

<img src='img/img_10.png'>

#### 1º passo:
procuramos pelo elemento select -> <select><option>opção 1</option><option>opção 2</option><option>opção 3</option></select>

<img src='img/img_8.png'>

O primeiro passo é encontrar e referenciar o elemento, temos algumas formas de fazer isto: <br><br>
find_element_by_id<br>
find_element_by_name<br>
find_element_by_xpath<br>
find_element_by_link_text<br>
find_element_by_partial_link_text<br>
find_element_by_tag_name<br>
find_element_by_class_name<br>
find_element_by_css_selector<br><br>

qualquer item nesta lista vai retornar o <strong>primeiro</strong> elemento que bater com a pesquisa,<br>
ou então posso pedir para retornar a <strong>lista</strong> de elementos que se encaixa com a pesquisa:<br><br>


find_elements_by_name<br>
find_elements_by_xpath<br>
find_elements_by_link_text<br>
find_elements_by_partial_link_text<br>
find_elements_by_tag_name<br>
find_elements_by_class_name<br>
find_elements_by_css_selector<br>


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

# driver = webdriver.Chrome('chromedrivers/Versão 75/Windows/chromedriver.exe', chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)

driver.get(url)
time.sleep(15)

#Podemos utilzar diferentes formas de selecionar o elemento de lista:

# lista = Select(driver.find_element_by_xpath('//*[@id="tipoImovelNovo"]'))
#ou
lista = Select(driver.find_element_by_xpath('//*[@id="l-select2"]'))
#ou
#lista = Select(driver.find_element_by_css_selector('select.select-novo'))
#ou
#lista = Select(driver.find_element_by_id('tipoImovelNovo'))
#ou
#lista = Select(driver.find_element_by_class_name('select-novo'))


### Sobre o Xpath

No Xpath existem vários tipos de nodos: <br>
elemento html, atributo, texto, entre outros...

##### exemplo:

<img src='img/img_11.png'>

<img src='img/img_12.png'>

#### sintaxe e exemplos:

<img src='img/img_13.png'>

<img src='img/img_14.png'>

<img src='img/img_15.png'>

Para ver mais: https://www.w3schools.com/xml/xpath_syntax.asp

Uma maneira fácil de descobrir o xpath:

<img src='img/img_16.png'>

In [ ]:
for item in lista.options:
    print(item.text)

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(15)

lista = Select(driver.find_element_by_xpath('//*[@id="l-select2"]'))


#maneiras diferentes de navegar pela lista:
#index
lista.select_by_index(2)
time.sleep(5)
#texto visível (de fato o que aparece ali)
lista.select_by_visible_text('Casa de Condomínio')
#pelo valor atribuído àquela opção (dispinível no html)
# time.sleep(5)
# lista.select_by_value('apartamento-padrao')

Outra maneira de navegar:

In [ ]:
l = ['Casa de Condomínio','Flat','Cobertura', 'Loft']

# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(8)

lista = Select(driver.find_element_by_xpath('//*[@id="l-select2"]'))

for item in l:
    lista.select_by_visible_text(item)
    time.sleep(3)

Navegando e captando os dados da págna:

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(15)

Select(driver.find_element_by_xpath('//*[@id="l-select2"]')).select_by_visible_text('Flat')

time.sleep(5)

page_source = driver.page_source

In [ ]:
parsed_content = BeautifulSoup(page_source, 'html.parser')
parsed_content

In [ ]:
parsed_content.find("div", {'class':'card-container'})

In [ ]:
parsed_content.find("div", {'class':'card-container'}).findAll('div', class_='simple-card__description')

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(15)

In [ ]:
quartos=driver.find_elements_by_css_selector('li.js-bedrooms')

In [ ]:
quartos[0].text

In [ ]:
quartos[2].text

In [ ]:
for q in quartos:
    v=q.text
    if v=='3':
        b=q

b.click()

## Daqui para baixo o conteúdo mudou de forma que o código não funciona mais

In [ ]:
botoes = parsed_content.find("div", {'class':'card-container'}).findAll('button')

In [ ]:
btns = []
for btn in botoes:
    item = btn.get('value')
    btns.append(item)

In [ ]:
btns

In [ ]:
slc = parsed_content.find("div", {'class':'card-container'}).find('button', {'class':'button active'}).get('value')
slc

In [ ]:
btns.remove(slc)

In [ ]:
btns

In [ ]:
# driver = webdriver.Chrome(path, chrome_options=options)
driver = webdriver.Firefox(executable_path=path, firefox_options=options)


driver.get(url)
time.sleep(2)

for n in btns:
    driver.find_element_by_xpath("//button[@value='" + n + "']").click()
    time.sleep(5)

In [ ]:
bairros = ['pinheiros','perdizes']

In [ ]:
url = 'https://www.zapimoveis.com.br/venda/apartamentos/sp+sao-paulo+zona-sul+moema/'

In [ ]:
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(path, chrome_options=options)
driver.get(url)
time.sleep(2)

for n in bairros:
    busca = driver.find_element_by_id('location')
    busca.send_keys(n)
    busca.send_keys(Keys.RETURN)
    time.sleep(5)
    
busca.send_keys('escrevi algo errado')
time.sleeo(5)
busca.clear()

Compilando tudo o que vimos até aqui: 

In [ ]:
url = 'https://www.zapimoveis.com.br/venda/apartamentos'

driver = webdriver.Chrome(path, chrome_options=options)
driver.get(url)
time.sleep(5)

lista = Select(driver.find_element_by_xpath('//*[@id="tipoImovelNovo"]'))
lista.select_by_index(5)
time.sleep(5)
lista.select_by_visible_text('Casa de Condomínio')
time.sleep(5)
lista.select_by_value('apartamento-padrao')
time.sleep(5)

l = ['Casa de Condomínio','Flat','Cobertura']

for item in l:
    lista.select_by_visible_text(item)
    time.sleep(5)
    
page_source = driver.page_source

parsed_content = BeautifulSoup(page_source, 'html.parser')

botoes = parsed_content.find("div", {'id':'divTransacao'}).findAll('button')

btns = []
for btn in botoes:
    item = btn.get('value')
    btns.append(item)
    
slc = parsed_content.find("div", {'id':'divTransacao'}).find('button', {'class':'button active'}).get('value')

btns.remove(slc)

for n in btns:
    driver.find_element_by_xpath("//button[@value='" + n + "']").click()
    time.sleep(5)
    
bairros = ['pinheiros','moema']

for n in bairros:
    busca = driver.find_element_by_id('location')
    busca.send_keys(n)
    busca.send_keys(Keys.RETURN)
    time.sleep(5)
    
busca.send_keys('escrevi algo errado')
time.sleeo(5)
busca.clear()

In [ ]:
url = 'http://www.ipea.gov.br/ipeageo/bases.html'

In [ ]:
conteudo = get(url).content

In [ ]:
parsed_content = BeautifulSoup(conteudo, 'html.parser')

In [ ]:
links = parsed_content.find('div', {'id':'editavel'}).find_all('li')

In [ ]:
links_download = []
for link in links:
    if link.find('a') is not None:    
        #l = link.a.get('href')
        l = link.a.text.strip()
        links_download.append(l)

In [ ]:
links_download[:2]

In [ ]:
prefs = {"download.default_directory" : r"C:\Users\victo\Desktop\Python\Selenium\Repositorio"}
options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(path, chrome_options=options)
driver.get(url)
time.sleep(5)

for link in links_download[:2]:
    driver.find_element_by_link_text(link).click()

In [ ]:
import os

path = r"C:\Users\victo\Desktop\Python\Selenium\Repositorio"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.xls' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)


In [ ]:
import pandas as pd

In [ ]:
pd.read_excel(files[0])